In [2]:
import pandas as pd
import xgboost as xgb
import shap
import dask.dataframe as dd
import os
import fsspec
import xgboost as xgb


In [ ]:
# Kombinierte Liste der zu entfernenden Spalten
columns_to_drop = [
    'ClmDiagnosisCode_1_Most_Frequent',
    'ClaimDuration_mean',
    'ChronicCond_Cancer_mean',
    'State_mode',
    'County_mode',
    'OPAnnualReimbursementAmt_mean',
    'OPAnnualReimbursementAmt_max',
    'perc_allocated_used',
    'DeductibleAmtPaid_sum'
]

# Spalten aus df_train_processed entfernen
df_train_processed_clean = df_train_processed.drop(columns=columns_to_drop)


In [3]:
# S3-Modellpfad
s3_path = "s3://medicare-fraud-data-25-05-2025/models/xgb_fraud_modelF.json"

# Lokaler Speicherort
local_model_dir = "/home/ec2-user/HIFP25/notebooks/"
os.makedirs(local_model_dir, exist_ok=True)
local_model_path = os.path.join(local_model_dir, "xgb_fraud_modelF.json")

# Download von S3 nach lokal
fs = fsspec.filesystem("s3")

with fs.open(s3_path, "rb") as s3_file:
    with open(local_model_path, "wb") as local_file:
        local_file.write(s3_file.read())

print(f"✅ Modell erfolgreich gespeichert unter: {local_model_path}")


✅ Modell erfolgreich gespeichert unter: /home/ec2-user/HIFP25/notebooks/xgb_fraud_modelF.json


In [4]:
import pandas as pd
import s3fs

# S3 Bucket + Ordner
bucket_path = "s3://medicare-fraud-data-25-05-2025/processedF/train/"

# S3-Dateisystem initialisieren
fs = s3fs.S3FileSystem()

# Alle CSV-Dateien im Ordner listen
csv_files = fs.glob(f"{bucket_path}*.csv")

# Alle CSV-Dateien in einen DataFrame laden und zusammenfügen
df_list = [pd.read_csv(f"s3://{file}", storage_options={"anon": False}) for file in csv_files]
X = pd.concat(df_list, ignore_index=True)

print("✅ CSV-Dateien erfolgreich geladen:", len(csv_files))


✅ CSV-Dateien erfolgreich geladen: 1


In [6]:
X.head(5)

,Race_mode,State_mode,County_mode,ClmAdmitDiagnosisCode_Most_Frequent,ClmDiagnosisCode_1_Most_Frequent,ClmDiagnosisCode_2_Most_Frequent,ClmDiagnosisCode_3_Most_Frequent,AttendingPhysician_Most_Frequent,OperatingPhysician_Most_Frequent,OtherPhysician_Most_Frequent,...,Avg_Deductible_Amt_Paid_Per_Provider,Avg_InscClaimAmtReimbursed_Per_Provider,perc_allocated_used,prv_avg_claims,prv_avg_claim_cost_indicator,prv_avg_claims_indicator,avg_cost_per_claim,perc_chronic_alz,Provider,PotentialFraud
0,0,9,19,191,277,264,694,2657,0,0,...,-0.375682,-0.423120,0.132287,-0.539095,-0.156164,-0.539095,-0.428244,-0.487604,PRV52145,0
1,0,32,3,768,277,264,142,289,0,0,...,0.065578,-0.133897,-0.713648,-0.366773,-0.327462,-0.366773,-0.417469,0.229200,PRV55104,0
2,0,32,64,472,11,264,271,4780,0,0,...,1.397284,1.513495,0.589595,-0.431634,6.155988,-0.431634,1.702587,0.036966,PRV54894,1
3,0,32,3,740,276,264,271,1471,0,0,...,-0.226785,0.031728,-0.199320,-0.451401,-0.298733,-0.451401,-0.436006,-0.198213,PRV54927,0
4,0,33,2,768,276,264,271,1127,0,0,...,-0.255899,-0.235811,-0.235060,-0.075958,8.119058,-0.075958,-0.306348,-0.075852,PRV55215,1


In [14]:
# Kombinierte Liste der zu entfernenden Spalten
columns_to_drop = [
    'ClmDiagnosisCode_1_Most_Frequent',
    'ClaimDuration_mean',
    'ChronicCond_Cancer_mean',
    'State_mode',
    'County_mode',
    'OPAnnualReimbursementAmt_mean',
    'OPAnnualReimbursementAmt_max',
    'perc_allocated_used',
    'DeductibleAmtPaid_sum',
    'avg_cost_per_claim',
    'PotentialFraud',
    'ClmDiagnosisCode_3_Most_Frequent',
    'perc_chronic_alz',
    'prv_avg_claims',
    'ClmDiagnosisCode_2_Most_Frequent',
    'Provider',
    'AttendingPhysician_Most_Frequent',
    'ClmAdmitDiagnosisCode_Most_Frequent'
]

# Spalten aus X entfernen
X = X.drop(columns=columns_to_drop, errors='ignore')



In [15]:
import xgboost as xgb
import pandas as pd
import shap
import s3fs

# 1. Lade das Modell
model = xgb.Booster()
model.load_model(local_model_path)
print("✅ Modell erfolgreich geladen")

# 2. Lade CSV-Dateien aus S3 (alle *.csv im train-Ordner)
fs = s3fs.S3FileSystem()  # ggf. anon=False falls S3 privat
file_list = fs.glob("medicare-fraud-data-25-05-2025/processedF/train/*.csv")

if not file_list:
    raise FileNotFoundError("⚠️ Keine CSV-Dateien im S3-Pfad gefunden!")

# CSV-Dateien laden und zusammenführen
dfs = [pd.read_csv(f"s3://{file}") for file in file_list]
X = pd.concat(dfs, ignore_index=True)
print(f"✅ {len(X)} Zeilen geladen")




✅ Modell erfolgreich geladen
✅ 5410 Zeilen geladen


In [16]:
model = xgb.Booster()
model.load_model(local_model_path)

model_features = model.feature_names
print("✅ Model expects:", model_features)


✅ Model expects: ['Race_mode', 'OperatingPhysician_Most_Frequent', 'OtherPhysician_Most_Frequent', 'ClaimDuration_sum', 'ClaimDuration_mean', 'ClaimDuration_std', 'ClaimDuration_max', 'ClaimDuration_min', 'HospitalDuration_sum', 'HospitalDuration_mean', 'HospitalDuration_std', 'HospitalDuration_max', 'HospitalDuration_min', 'DeductibleAmtPaid_sum', 'DeductibleAmtPaid_mean', 'DeductibleAmtPaid_std', 'DeductibleAmtPaid_max', 'DeductibleAmtPaid_min', 'IPAnnualReimbursementAmt_mean', 'IPAnnualReimbursementAmt_max', 'IPAnnualDeductibleAmt_mean', 'IPAnnualDeductibleAmt_max', 'OPAnnualReimbursementAmt_mean', 'OPAnnualReimbursementAmt_max', 'OPAnnualDeductibleAmt_mean', 'OPAnnualDeductibleAmt_max', 'ChronicCond_Alzheimer_sum', 'ChronicCond_Alzheimer_mean', 'ChronicCond_Heartfailure_sum', 'ChronicCond_Heartfailure_mean', 'ChronicCond_KidneyDisease_sum', 'ChronicCond_KidneyDisease_mean', 'ChronicCond_Cancer_sum', 'ChronicCond_Cancer_mean', 'ChronicCond_ObstrPulmonary_sum', 'ChronicCond_ObstrPulm

In [18]:
X_columns = list(X.columns)

# What's missing in the input but expected by the model
missing = list(set(model_features) - set(X_columns))
extra = list(set(X_columns) - set(model_features))

print("❌ Missing columns:", missing)
print("⚠️ Extra columns:", extra)


❌ Missing columns: []
⚠️ Extra columns: ['avg_cost_per_claim', 'PotentialFraud', 'State_mode', 'ClmDiagnosisCode_3_Most_Frequent', 'perc_chronic_alz', 'prv_avg_claims', 'ClmDiagnosisCode_2_Most_Frequent', 'Provider', 'AttendingPhysician_Most_Frequent', 'ClmDiagnosisCode_1_Most_Frequent', 'County_mode', 'ClmAdmitDiagnosisCode_Most_Frequent']


In [19]:
# Optional: sichere dir die Provider ID (wird später gebraucht)
provider_ids = X["Provider"]

# Entferne nicht-numerische Spalte(n) vor der DMatrix-Erstellung
X_model_input = X.drop(columns=["Provider"])  # ggf. weitere object-Spalten auch prüfen

# 3. XGBoost DMatrix für Prediction
dmat = xgb.DMatrix(X_model_input)

# 4. Vorhersagen holen
probs = model.predict(dmat)

# 5. SHAP-Werte berechnen
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# 6. Top 3 SHAP-Werte pro Zeile extrahieren
top3_shap_values = []
top3_features = []

for row in shap_values:
    top_indices = abs(row).argsort()[-3:][::-1]
    top_shaps = row[top_indices]
    top_feats = X.columns[top_indices]
    top3_shap_values.append(top_shaps)
    top3_features.append(top_feats)

# 7. Ergebnisse zusammenbauen
results = pd.DataFrame({
    "Provider": provider_ids,
    "Fraud_Probability": probs,
    "Top1_Feature": [f[0] for f in top3_features],
    "Top1_SHAP": [s[0] for s in top3_shap_values],
    "Top2_Feature": [f[1] for f in top3_features],
    "Top2_SHAP": [s[1] for s in top3_shap_values],
    "Top3_Feature": [f[2] for f in top3_features],
    "Top3_SHAP": [s[2] for s in top3_shap_values],
})

# Optional: Prozentformat
results["Fraud_Probability_Percent"] = results["Fraud_Probability"] * 100

# Vorschau
print(results.head())

# Optional speichern:
results.to_csv("fraud_predictions_with_shap.csv", index=False)
print("✅ Ergebnisse gespeichert in fraud_predictions_with_shap.csv")


ValueError: feature_names mismatch: ['Race_mode', 'OperatingPhysician_Most_Frequent', 'OtherPhysician_Most_Frequent', 'ClaimDuration_sum', 'ClaimDuration_mean', 'ClaimDuration_std', 'ClaimDuration_max', 'ClaimDuration_min', 'HospitalDuration_sum', 'HospitalDuration_mean', 'HospitalDuration_std', 'HospitalDuration_max', 'HospitalDuration_min', 'DeductibleAmtPaid_sum', 'DeductibleAmtPaid_mean', 'DeductibleAmtPaid_std', 'DeductibleAmtPaid_max', 'DeductibleAmtPaid_min', 'IPAnnualReimbursementAmt_mean', 'IPAnnualReimbursementAmt_max', 'IPAnnualDeductibleAmt_mean', 'IPAnnualDeductibleAmt_max', 'OPAnnualReimbursementAmt_mean', 'OPAnnualReimbursementAmt_max', 'OPAnnualDeductibleAmt_mean', 'OPAnnualDeductibleAmt_max', 'ChronicCond_Alzheimer_sum', 'ChronicCond_Alzheimer_mean', 'ChronicCond_Heartfailure_sum', 'ChronicCond_Heartfailure_mean', 'ChronicCond_KidneyDisease_sum', 'ChronicCond_KidneyDisease_mean', 'ChronicCond_Cancer_sum', 'ChronicCond_Cancer_mean', 'ChronicCond_ObstrPulmonary_sum', 'Ch